## MaskContours-Misclassification

MaskContours-Testing discovered a few remaining issues, one of which looks like a systematic misclassification of digits where digit N is misclassified as digit N - 1. The working theory is that the atlas template match is *just* to the left of the intended digit's column. If this is the case, should be an easy fix.

In [ ]:
mask_dir = r'E:\Ultimate\day1_streams\masks'

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import cv2
import xml.etree.ElementTree as ET

from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [ ]:
test_image_path = 'E:/Ultimate/stream_data/masks/102_58.1.png'
image_mask_dir = 'E:/Ultimate/stream_data/masks'

In [ ]:
def test_and_plot_image(test_image_path):
    img_rgb = cv2.cvtColor(cv2.imread(test_image_path), cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    img_inrange = cv2.inRange(img, (0, 0, 0), (180, 255, 80))
    img_inverted = cv2.bitwise_not(img_inrange)
    
    _, ax = plt.subplots(1, 2)
    ax[0].imshow(img_rgb)
    ax[1].imshow(img_inverted)
    
    return img_inverted

In [ ]:
# TODO: Measure perf
#img_inverted = test_and_plot_image(test_image_path)

In [ ]:
def extract_blobs(binary_image):
    connectivity = 4
    output = cv2.connectedComponentsWithStats(binary_image, connectivity, cv2.CV_8S)
    
    num_labels = output[0]
    labels = output[1]
    stats = output[2]
    centroids = output[3]
    
    blobs = []
    xcoords = []
    for label in range(num_labels):
        stats_indices = [cv2.CC_STAT_LEFT, cv2.CC_STAT_WIDTH, cv2.CC_STAT_TOP, cv2.CC_STAT_HEIGHT]
        xmin, width, ymin, height = [stats[label][i] for i in stats_indices]
        
        xmax = xmin + width
        ymax = ymin + height

        blob = np.where(labels[ymin:ymax, xmin:xmax] == label, 255, 0).astype('uint8')
        blobs.append(blob)
        xcoords.append(xmin)
        
    return blobs, xcoords

In [ ]:
# Make an atlas from the digits that I have at this directory
digits_dir = r'E:/Ultimate/stream_data/digits'
digits_index = [[] for _ in range(10)]
stats_array = []

for d in os.listdir(digits_dir):
    full_root = os.path.join(digits_dir, d)
    
    for f in os.listdir(full_root):
        full_path = os.path.join(full_root, f)
        
        i = int(d)
        img = cv2.imread(full_path)
        binary_image = cv2.inRange(img, (40, 40, 40), (255, 255, 255))
        digits_index[i].append(binary_image)
        
        stats_array.append(digits_index[i][-1].shape[:2])
        
for i in range(10):
    print("{} : {}".format(i, len(digits_index[i])))

In [ ]:
cell_dim = 32
#min_rows = np.min(np.array([len(digits_index[i]) for i in range(10)]))
min_rows = 10
print("Min rows = {}".format(min_rows))
atlas_height = min_rows * cell_dim
atlas_width  = cell_dim * 10

atlas = np.zeros((atlas_height, atlas_width)).astype('uint8')

for digit in range(10):
    np.random.shuffle(digits_index[digit])
    for i, img in enumerate(digits_index[digit][:min_rows]):
        xmin, ymin = digit * cell_dim, i * cell_dim
        xmax, ymax = xmin + cell_dim, ymin + cell_dim
        
        cell_img = cv2.resize(img, (cell_dim, cell_dim))
        atlas[ymin:ymax, xmin:xmax] = cell_img
        
cv2.imwrite(r'C:\users\phyto\desktop\atlas2.png', atlas)

In [ ]:
gt_directories = [
    r'E:/Ultimate/stream_data/corpus_2',
    r'E:/Ultimate/stream_data/corpus_3', 
    r'E:/Ultimate/stream_data/corpus_4', 
    r'E:/Ultimate/stream_data/corpus_5',
    r'E:/Ultimate/stream_data/corpus_6',
    r'E:/Ultimate/stream_data/corpus_7'
]

def get_regions_from_path(img_path):
    # We're assuming the GT annotations file is right alongside 
    # this image.
    
    regions = []
    annotations_path = img_path.replace(".png", ".xml")
    if not os.path.exists(annotations_path):
        return regions
    
    with open(annotations_path, 'r') as fr:
        tree = ET.parse(fr)
        root = tree.getroot()
        
        objects = root.findall('object')
        if objects is None or len(objects) == 0:
            return regions
        else:
            img_rgb = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            for obj in objects:
                cls = obj.find('name').text
                if ':' in cls:
                    continue
                
                xmlbox = obj.find('bndbox')
                
                xmin, xmax = int(xmlbox.find('xmin').text), int(xmlbox.find('xmax').text)
                ymin, ymax = int(xmlbox.find('ymin').text), int(xmlbox.find('ymax').text)
                
                regions.append((img_rgb[ymin:ymax, xmin:xmax], cls))
                
    return regions

In [ ]:
images = []
gtvalues = []
for gt_dir in gt_directories:
    print(gt_dir)
    for filename in [f for f in os.listdir(gt_dir) if f[-4:].lower() == ".png"]:
        full_path = os.path.join(gt_dir, filename)
        
        regions = get_regions_from_path(full_path)
        images.extend([r[0] for r in regions])
        gtvalues.extend([r[1] for r in regions])

In [ ]:
digits_path = r'E:/Ultimate/stream_data/digits'
#nondigits_path = r'C:\Users\phyto\Desktop\ultimate\nondigits'

def get_image_stats(path):
    all_stats = []
    for root, _, files in os.walk(path):
        for f in files:
            if f[-4:] == ".png":
                full_path = os.path.join(root, f)
                im = cv2.cvtColor(cv2.imread(full_path), cv2.COLOR_BGR2GRAY)

                stats = (im.shape[1], im.shape[0], cv2.countNonZero(im), os.path.basename(root))

                all_stats.append(stats)
                
    return all_stats

digits_stats = get_image_stats(digits_path)
#nondigits_stats = get_image_stats(nondigits_path)

In [ ]:
def img_area(im):
    return im.shape[0] * im.shape[1]

def filter_results(blobs_to_filter):
    blobs_by_area = sorted(blobs_to_filter, key = lambda x: img_area(x[0]), reverse=True)
    largest = blobs_by_area[0]
    
    #print([(blob[0].shape[0] / largest[0].shape[0]) for blob in blobs_by_area])
    
    large_blobs = [('big', blob) for blob in blobs_by_area if (blob[0].shape[0] / largest[0].shape[0]) > 0.87]
    large_blobs.extend([('small', blob) for blob in blobs_by_area if (blob[0].shape[0] / largest[0].shape[0]) <= 0.445 and (blob[0].shape[0] / largest[0].shape[0]) >= 0.355])
    
    return sorted(large_blobs, key = lambda x: x[1][2]), [(blob[0].shape[0] / largest[0].shape[0]) for blob in blobs_by_area]

def position_digits(blobs, gtvalue):
    blob_positions = [-1] * 4
    
    big_blobs = [blob[1] for blob in blobs if blob[0] == 'big']
    if len(big_blobs) == 0:
        return blob_positions
    elif len(big_blobs) > 3:
        big_blobs = big_blobs[-3:]
    
    small_blobs = [blob[1] for blob in blobs if blob[0] == 'small']
    if len(small_blobs) > 0:
        blob_positions[3] = small_blobs[0][1]
        
    for i, b in enumerate(reversed(big_blobs)):
        target_index = 2 - i
        blob_positions[target_index] = b[1]

    gt_positions = [-1] * 4
    for i, d in enumerate(reversed(gtvalue.replace('.', ''))):
        index = 3 - i
        gt_positions[index] = int(d)
            
    return blob_positions, gt_positions

def confusion_results(blob_positions, gt_positions, confusion_matrix = None):
    if confusion_matrix is None:
        confusion_matrix = np.zeros((10, 10))
        
    for i in range(len(gt_positions)):
        c_i, c_j = gt_positions[i], blob_positions[i]
        
        if c_i >= 0 and c_j >= 0:
            confusion_matrix[c_j, c_i] += 1
            
    return confusion_matrix

def missing_results(blob_positions, gt_positions):
    pass

def is_bad(blob_positions, gt_positions):
    #
    # Just check the 'big' digits
    #
    zipped = zip(blob_positions[:3], gt_positions[:3])
    return not all([i == j for (i, j) in zipped])

In [ ]:
atlas_copy = atlas.copy()

def get_blobs(images):
    good_regions = []
    bad_regions = []
    confusion_matrix = np.zeros((10, 10))
    for j, image in enumerate(images):
        gtvalue = gtvalues[j]
        
        hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        #img_inrange = cv2.inRange(hsv_image, (0, 0, 0), (180, 255, 80))
        img_inrange = cv2.inRange(hsv_image, (0, 0, 0), (180, 255, 120))
        img_inverted = cv2.bitwise_not(img_inrange)

        blobs, xcoords = extract_blobs(img_inverted)
        blobs_to_render = []
        rejected_blobs = []
        scores = []# [(blob[0].shape[0] / largest[0].shape[0]) for blob in sorted(blobs, key=lambda x : img_area(x))]

        for i, blob in enumerate(blobs):
            if np.sum(blob) < 3000:
                rejected_blobs.append((blob, -1.0))
                continue

            blob_cell = cv2.resize(blob, (cell_dim, cell_dim))
            response = cv2.matchTemplate(blob_cell, atlas, cv2.TM_CCORR_NORMED)
            _, max_val, _, max_loc = cv2.minMaxLoc(response)

            if max_val > 0.92:
                digit = int(max_loc[0] + cell_dim / 2) // cell_dim

                #if digit == 0:
                #    cv2.circle(atlas_copy, max_loc + cell_dim / 2, 5, (0, 255, 0))
                #if digit == 1:
                #    cv2.circle(atlas_copy, max_loc, 5, (255, 0, 0))
                blobs_to_render.append((blob, digit, xcoords[i], max_val))
            else:
                rejected_blobs.append((blob, max_val))


        if len(blobs_to_render) == 0:
            bad_regions.append((image, blobs_to_render, None, None, gtvalue, rejected_blobs))
            continue
        
        blobs_to_render, _ = filter_results(blobs_to_render)
        bs, gs = position_digits(blobs_to_render, gtvalue)
        confusion_matrix = confusion_results(bs, gs, confusion_matrix)

        if is_bad(bs, gs):
            bad_regions.append((image, blobs_to_render, gs, bs, gtvalue, rejected_blobs, scores))
        else:
            good_regions.append((image, blobs_to_render, gs, bs, gtvalue, rejected_blobs, scores))
            
    return good_regions, bad_regions, confusion_matrix
    
def render_regions(regiondata):
    for r in regiondata:
        #score = compute_gt_score(blobs_to_render, )
        blobs_to_render = r[1]
        image           = r[0]
        gtvalue         = r[4]
        #rejected_blobs  = r[5]
        rejected_blobs  = [] # disable for now
        scores          = [] if len(r) < 7 else r[6]

        if len(blobs_to_render) < 1:
            plt.imshow(image)
            continue

        print(scores)
        _, ax = plt.subplots(1, len(blobs_to_render) + 1, figsize=(20, 10))
        ax[0].imshow(image)
        ax[0].set_title("GT value = {}".format(gtvalue))

        widths = []
        heights = []

        for i in range(1, len(blobs_to_render) + 1):
            digit = blobs_to_render[i - 1][1][1]
            blob  = blobs_to_render[i - 1][1][0]
            max_val = blobs_to_render[i - 1][1][3]

            widths.append(blob.shape[1])
            heights.append(blob.shape[0])

            ax[i].set_title("Classified as: {} ({:2f})".format(digit, max_val))
            ax[i].imshow(blob)
        
        _, ax = plt.subplots(1, len(rejected_blobs), figsize=(20, 10))
        for i, rb in enumerate(rejected_blobs):
            blob = rb[0]
            score = rb[1]
            
            ax[i].imshow(blob)
            ax[i].set_title("Score = {}".format(score))

        plt.show()
        plt.close('all')

In [ ]:
processed_images = []
N = 32
for image in images:
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #quantized = np.floor(gray*(N/255)*(255/N)).astype('uint8')
    canny = cv2.Canny(gray, 80, 210) 
    
    #kernel = np.ones((2,2),np.uint8)
    #canny = dilation = cv2.dilate(canny,kernel,iterations = 1)
    
    new_image = image.copy()
    new_image[np.where(canny > 0)] = (0, 0, 0)
    processed_images.append(new_image) 

In [ ]:
#good_regions, bad_regions, confusion_matrix = get_blobs(images)
good_regions, bad_regions, confusion_matrix = get_blobs(processed_images)

In [ ]:
confusion_matrix

In [ ]:
#plt.imshow(atlas_copy)

In [ ]:
# 519; 41 @ 110
# 522; 38 @ 120
# 520, 40 @ 125
# 518; 42 @ 130
# 515; 45 @ 140
print(len(good_regions))
print(len(bad_regions))
print(len(good_regions) / (len(good_regions) + len(bad_regions)))

# with and w/o oquantized, + dilation: 523; 37, 0.91 thresh
# no quantization, no dilation: 523, 37, 0.91 thresh
# no quantization, no dilation: 528, 32, 0.92 thresh

In [ ]:
#render_regions(good_regions)

In [ ]:
#render_regions(bad_regions)

In [ ]:
dump_dir = r'C:\Users\phyto\Desktop\temp\thresholding_experiment'

for i, r in enumerate(bad_regions):
    image = cv2.cvtColor(r[0], cv2.COLOR_BGR2RGB)
    
    full_path = os.path.join(dump_dir, str(i) + '.png')
    # disable for now
    cv2.imwrite(full_path, image)